# 문장 유사도를 이용한 ChatBot 개발

## 데이터 생성

In [1]:
### 필요한 라이브러리 / 함수 임폴트
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [2]:
### 챗봇 생성용 데이터 불러오기

# 파일 경로 설정
file_path='D:\Code\DataSets/ChatBotData.csv'

# pd.read_csv() 사용 --> DataFrame 생성
df = pd.read_csv(file_path)

# 결과 확인하기
print(df)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


In [3]:
### 사전 학습된 한국어 SentenceBERT 모델 생성
model_name = 'ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

In [4]:
### 모델 테스트 : 텍스트 --> 임베딩

# 테스트 문장 생성
sentence = "본 발명은 바위수염 추출물을 포함하는 항비만용 조성물 및 그 제조방법에 관한 것이다"

# model.encode(sentence) 사용
embedding = model.encode(sentence)

# 문장 임베딩 결과 확인
print(f'임베딩 벡터의 모양 확인 : {embedding.shape}')
print('-'*80)
print(f'임베딩 벡터 확인: \n{embedding}')

임베딩 벡터의 모양 확인 : (768,)
--------------------------------------------------------------------------------
임베딩 벡터 확인: 
[ 2.47432545e-01  3.19152832e-01 -2.05433086e-01  4.23581570e-01
 -5.40823638e-01  6.10795736e-01 -2.74022907e-01 -5.51382422e-01
 -2.76437491e-01 -1.81457639e-01  4.33313996e-01 -4.46095854e-01
  7.90904537e-02 -9.16764736e-01 -2.76828259e-01 -7.00665731e-03
  2.35826466e-02  6.20459974e-01  7.21695423e-01 -7.87704706e-01
 -3.69594634e-01  3.44950855e-01  4.21566725e-01 -4.31745797e-01
 -1.84948713e-01  1.36051178e-01  1.34623542e-01 -5.27529597e-01
 -2.24852368e-01 -4.65934604e-01  5.44648409e-01 -1.61633119e-01
  1.30852982e-01 -2.97314227e-01 -3.55800748e-01  3.07409726e-02
 -7.84511864e-01 -7.73454383e-02  1.87774405e-01  4.93741259e-02
  1.90536708e-01 -1.20568149e-01  3.50683361e-01  6.41438365e-02
  1.39915302e-01 -1.95609972e-01  9.60729942e-02  1.94002151e-01
  3.45328361e-01  2.39388183e-01 -4.80340809e-01  2.85571158e-01
  2.81684458e-01 -7.72601545e-01 -1.546

In [5]:
### 문장 임베딩 생성

"""
### apply(lambda x : model.encode(x)) 적용
"""

# 임베딩 벡터 생성
embeddings = df.loc[:, 'Q'].apply(lambda x: model.encode(x)).values

# 결과 확인하기
print(f'문장 임베딩 벡터 전체의 모양 : {embeddings.shape}')
print('-'*80)
print(f'전체 문장 임베딩 벡터 출력 : \n{embeddings}')

문장 임베딩 벡터 전체의 모양 : (11823,)
--------------------------------------------------------------------------------
전체 문장 임베딩 벡터 출력 : 
[array([-1.38743147e-01,  1.51097598e-02,  5.28436124e-01,  7.21492290e-01,
        -3.91227007e-01, -7.02952206e-01, -8.01476419e-01,  4.91261810e-01,
         2.17922330e-02,  3.80374223e-01, -6.92348897e-01, -1.11403167e-01,
         2.16700539e-01,  3.51305723e-01,  2.36636028e-01,  6.80069447e-01,
         1.71047077e-01, -5.12855589e-01, -1.09594464e-01,  6.90377280e-02,
         2.15584729e-02, -7.99720705e-01, -1.68677211e-01,  6.13967121e-01,
         3.97408336e-01,  4.25235122e-01,  5.24639077e-02,  2.25223854e-01,
        -3.69577497e-01, -3.86963755e-01, -6.03432000e-01,  4.99111801e-01,
        -1.14106156e-01, -1.77366421e-01,  1.42669916e-01, -5.48125505e-01,
         4.89641547e-01, -2.73698539e-01,  5.87849677e-01,  3.24973911e-01,
         5.86492568e-02,  7.93394268e-01, -1.63209841e-01,  6.36749566e-02,
        -2.92038769e-01, -4.99974519

In [6]:
### 저장된 임베딩 벡터 불러오기

# 파일 경로 설정하기
file_path='D:\Code\DataSets/chatbot_embeddings.npy'

# np.load() 사용
embeddings = np.load(file_path, allow_pickle=True)

# 결과 확인하기
print(f'문장 임베딩 벡터 전체의 모양 : {embeddings.shape}')
print('-'*80)
print(f'전체 문장 임베딩 벡터 출력 : \n{embeddings}')

문장 임베딩 벡터 전체의 모양 : (11823,)
--------------------------------------------------------------------------------
전체 문장 임베딩 벡터 출력 : 
[array([-1.38743490e-01,  1.51098920e-02,  5.28436601e-01,  7.21491992e-01,
        -3.91226649e-01, -7.02952325e-01, -8.01476300e-01,  4.91262317e-01,
         2.17927936e-02,  3.80374193e-01, -6.92348778e-01, -1.11402906e-01,
         2.16700792e-01,  3.51305634e-01,  2.36635745e-01,  6.80069447e-01,
         1.71046853e-01, -5.12855828e-01, -1.09594338e-01,  6.90384209e-02,
         2.15584412e-02, -7.99720585e-01, -1.68677449e-01,  6.13967717e-01,
         3.97407800e-01,  4.25234914e-01,  5.24640083e-02,  2.25224003e-01,
        -3.69577408e-01, -3.86964351e-01, -6.03431880e-01,  4.99112129e-01,
        -1.14106648e-01, -1.77365944e-01,  1.42669722e-01, -5.48124969e-01,
         4.89641517e-01, -2.73698479e-01,  5.87850034e-01,  3.24973583e-01,
         5.86489625e-02,  7.93394864e-01, -1.63209572e-01,  6.36754707e-02,
        -2.92038769e-01, -4.99974966

In [7]:
### 챗봇 생성용 데이터 --> embedding이라는 새로운 컬럼 생성, 문장 임베딩 벡터 저장

# 새로운 컬럼 생성
df['embedding'] = embeddings

# 결과 확인하기
print(f'전체 데이터프레임 확인 : \n{df}')
print('-'*80)
print(f'embedding 컬럼의 0번째 성분 원소 확인 : \n{df.iloc[0, -1]}')

전체 데이터프레임 확인 : 
                             Q                         A  label  \
0                       12시 땡!                하루가 또 가네요.      0   
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0   
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0   
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0   
4                      PPL 심하네                눈살이 찌푸려지죠.      0   
...                        ...                       ...    ...   
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2   
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2   
11820              흑기사 해주는 짝남.                    설렜겠어요.      2   
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2   
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2   

                                               embedding  
0      [-0.13874349, 0.015109892, 0.5284366, 0.721492...  
1      [0.08192257, 0.30836895, 0.27681014, -0.060137...  
2 

## ChatBot 생성

In [8]:
### 두 문장의 텍스트 유사도 계산 함수 정의

# 필요한 함수 임폴트
from numpy import dot
from numpy.linalg import norm

# 사용자 정의 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

In [14]:
### 사용자 질문에 대한 응답 구현

'''
1. 사용자의 질문이 들어오면 해당 질문에 대한 문장 임베딩 값을 구한다.
2. 사용자 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 컬럼(df.loc[:, 'embedding'])에
저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 이용, 코사인 유사도를 계산
--> 코사인 유사도 값을 저장하는 컬럼(df.loc[:, 'score'])을 생성하고 값을 저장
3. 코사인 유사도 값이 가장 높은 질문 샘플을 찾는다.
4. 해당 질문 샘플과 짝이 되는 답변 샘플을 반환한다.
'''

# 사용자 정의 함수 생성
def chatbot(question):
    # 사용자 질문 문장 --> 문장 임베딩 벡터 구하기
    embedding = model.encode(question)
    # 코사인 유사도 계산 --> score 컬럼 생성, 코사인 유사도 값을 저장
    print(df.loc[:, 'embedding'].apply(lambda x: cos_sim(x, embedding)).values)
    df['score'] = df.loc[:, 'embedding'].apply(lambda x: cos_sim(x, embedding)).values
    # 코사인 유사도 값이 가장 큰 질문 샘플을 찾아서 해당 질문과 짝이 되는 답변 추출 = pandas.Series.idxmax
    
    return df.loc[df.loc[:,'score'].idxmax(), 'A'] # score컬럼에서 행 중에 값이 최대인 행이 나옴. 행조건.
    # print(df)
    
    

In [33]:
### 챗봇 성능 테스트(1)
text = input()

ans = chatbot(text)

print(ans)

[0.14931118 0.00390461 0.02318809 ... 0.01869551 0.19213721 0.25843996]
내일은 나을 거예요.
